In [1]:
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score
import tensorflow as tf
from transformers import TFAutoModel, AutoTokenizer, BertForPreTraining
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
import numpy as np
import time
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import AdaBoostClassifier
from tqdm import tqdm
from sklearn.metrics import ConfusionMatrixDisplay

C:\Users\zoika\anaconda3\envs\tf_gpu_38\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Διαβασμα του dataset με τα χαρακτηριστικά που έφτιαξα 


## Read the dataset with the characteristics i made 

In [2]:
path_train = "../../dataset/train_3.csv"
path_test = "../../dataset/test_3.csv"
df = pd.read_csv (path_train)
df_t = pd.read_csv (path_test)

In [3]:
# df_dropped = df.drop('ID', axis=1)
y_train = df['prediction']
x_train_1 = df.drop('prediction', axis=1)
x_train_1 = x_train_1.drop('interruption', axis=1)
x_train_1 = x_train_1.drop('uh', axis=1)
x_train_1 = x_train_1.drop('retrace_with_corr', axis=1)

In [4]:
# ids_test = df_t['ID'].tolist()
# df_test_dropped = df_t.drop('ID', axis=1)
y_test = df_t['prediction']
x_test_1 = df_t.drop('prediction', axis=1)
x_test_1 = x_test_1.drop('interruption', axis=1)
x_test_1 = x_test_1.drop('uh', axis=1)
x_test_1 = x_test_1.drop('retrace_with_corr', axis=1)

In [5]:
ids_train_1 = df['ID'].tolist()
ids_test_1 = df_t['ID'].tolist()

In [6]:
x_train_1

,ID,Difficulties,retrace_no_corr,gfi,ari,unique_words,text_len,stop_words
0,S097,0.018182,0.052632,0.160551,0.302301,0.051643,0.421053,0.540146
1,S080,0.020000,0.052632,0.043633,0.109326,0.146199,0.157895,0.520000
2,S086,0.053571,0.052632,0.153351,0.192617,0.074369,0.473684,0.535714
3,S110,0.072727,0.052632,0.059399,0.081080,0.065632,0.421053,0.572727
4,S082,0.050000,1.000000,0.189549,0.286042,0.028382,0.684211,0.540323
...,...,...,...,...,...,...,...,...
103,S076,0.018519,0.000000,0.138324,0.235141,0.074689,0.368421,0.529412
104,S077,0.018182,0.000000,0.152717,0.263396,0.041793,0.421053,0.564706
105,S070,0.000000,0.052632,0.195747,0.231222,0.076023,0.263158,0.472222
106,S073,0.054545,0.052632,0.116815,0.206553,0.064706,0.421053,0.515464


In [7]:
def metrics(y_pred, y_true):
  print("f1 score: ", f1_score(y_true, y_pred))
  print("accuracy: ", accuracy_score(y_true, y_pred))
  print("precision:", precision_score(y_true, y_pred, zero_division=1))
  print("recall: ", recall_score(y_true, y_pred, zero_division=1))

## Διάβασμα του 2ου dataset και χρήση του countvectorizer για την εξαγωγή χαρακτηριστικών

## Read dataset no.2, bert's dataset, and use a countvectorizer to extract features 

In [8]:
path_train = "../../dataset/train_bert_new_4.csv"
path_test = "../../dataset/test_bert_new_4.csv"

df_train = pd.read_csv(path_train)
df_test = pd.read_csv(path_test)

ids_train = df_train['ID'].tolist()
ids_test = df_test['ID'].tolist()

x_train = df_train['text'].tolist()
x_test = df_test['text'].tolist()
y_train = df_train['prediction'].tolist()
y_test = df_test['prediction'].tolist()

In [9]:
count_vect = CountVectorizer()

x_train_counts = count_vect.fit_transform(x_train)
x_test_counts = count_vect.transform(x_test)

max_features = max(x_train_counts.shape[1], x_test_counts.shape[1])
count_vect = CountVectorizer(max_features=max_features)

x_train_counts = count_vect.fit_transform(x_train)

x_test_counts = count_vect.transform(x_test)

print("x_train_counts shape:", x_train_counts.shape)
print("x_test_counts shape:", x_test_counts.shape)

x_train_counts shape: (108, 792)
x_test_counts shape: (48, 792)


In [10]:
scaler = MinMaxScaler()

x_train_counts = scaler.fit_transform(x_train_counts.toarray())

x_test_counts = scaler.transform(x_test_counts.toarray())

print("Normalized x_train_counts shape:", x_train_counts.shape)
print("Normalized x_test_counts shape:", x_test_counts.shape)


Normalized x_train_counts shape: (108, 792)
Normalized x_test_counts shape: (48, 792)


In [11]:
x_train_counts_df = pd.DataFrame(x_train_counts, columns=count_vect.get_feature_names_out())
ids_train_df = pd.DataFrame({'ID': ids_train})
x_train_counts_df = pd.concat([ids_train_df, x_train_counts_df], axis=1)


x_test_counts_df = pd.DataFrame(x_test_counts, columns=count_vect.get_feature_names_out())
ids_test_df = pd.DataFrame({'ID': ids_test})
x_test_counts_df = pd.concat([ids_test_df, x_test_counts_df], axis=1)

x_train_counts_df

,ID,about,above,across,action,active,activity,addition,after,again,...,wrong,yard,yeah,years,yes,yet,you,young,youngster,your
0,S097,0.00,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
1,S080,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
2,S086,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.142857,0.0,0.0,0.0
3,S110,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.428571,0.0,0.0,0.0
4,S082,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.428571,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103,S076,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.000000,0.0,0.0,0.0
104,S077,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
105,S070,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.2,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
106,S073,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.142857,0.0,0.0,0.0


## Συνδυασμός των 2 dataset 

## Combine the 2 datasets

In [12]:
print(type(x_train_1),"_", type(x_train_counts_df)) 
print(type(x_test_1),"_", type(x_test_counts_df))
import pandas as pd

# Assuming x_train_1 and x_train_counts_df both have a column named 'ID'
combined_df = pd.merge(x_train_1, x_train_counts_df, on='ID', how='inner')

# Assuming x_test_1 and x_test_counts_df both have a column named 'ID'
combined_df_t = pd.merge(x_test_1, x_test_counts_df, on='ID', how='inner')


<class 'pandas.core.frame.DataFrame'> _ <class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'> _ <class 'pandas.core.frame.DataFrame'>


In [13]:
combined_df_t

,ID,Difficulties,retrace_no_corr,gfi,ari,unique_words,text_len,stop_words,about,above,...,wrong,yard,yeah,years,yes,yet,you,young,youngster,your
0,S172,0.000000,0.000000,0.612036,0.755782,0.045183,0.444444,0.556818,0.00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.142857,0.0,0.0,0.0
1,S171,0.053571,0.142857,0.303268,0.413992,0.124722,0.611111,0.479167,0.00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
2,S174,0.051724,0.142857,0.562876,0.752689,0.043973,0.722222,0.513966,0.00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
3,S166,0.032258,0.285714,0.685238,0.838096,0.017976,0.944444,0.567951,0.25,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.142857,0.0,0.0,0.0
4,S160,0.039216,0.000000,0.597423,0.683945,0.132812,0.333333,0.509804,0.25,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
5,S161,0.090909,0.000000,0.311246,0.346557,0.056065,0.555556,0.500000,0.00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
6,S167,0.245902,0.571429,0.594103,0.702706,0.024637,0.888889,0.491468,0.00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.428571,0.0,0.0,0.0
7,S170,0.076923,0.000000,0.513828,0.911290,0.136483,0.388889,0.483333,0.00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
8,S168,0.120000,0.142857,0.414519,0.613126,0.117647,0.277778,0.516667,0.00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
9,S165,0.105263,0.285714,0.139961,0.046774,0.050667,0.666667,0.550847,0.25,0.0,...,0.0,0.0,0.2,0.0,1.0,0.0,0.000000,0.0,0.0,0.0


In [14]:
x_train = combined_df.drop('ID', axis=1)
x_test = combined_df_t.drop('ID', axis=1)

In [15]:
x_train

,Difficulties,retrace_no_corr,gfi,ari,unique_words,text_len,stop_words,about,above,across,...,wrong,yard,yeah,years,yes,yet,you,young,youngster,your
0,0.018182,0.052632,0.160551,0.302301,0.051643,0.421053,0.540146,0.00,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
1,0.020000,0.052632,0.043633,0.109326,0.146199,0.157895,0.520000,0.00,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
2,0.053571,0.052632,0.153351,0.192617,0.074369,0.473684,0.535714,0.25,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.142857,0.0,0.0,0.0
3,0.072727,0.052632,0.059399,0.081080,0.065632,0.421053,0.572727,0.00,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.428571,0.0,0.0,0.0
4,0.050000,1.000000,0.189549,0.286042,0.028382,0.684211,0.540323,0.00,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.428571,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103,0.018519,0.000000,0.138324,0.235141,0.074689,0.368421,0.529412,0.25,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.000000,0.0,0.0,0.0
104,0.018182,0.000000,0.152717,0.263396,0.041793,0.421053,0.564706,0.00,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
105,0.000000,0.052632,0.195747,0.231222,0.076023,0.263158,0.472222,0.25,0.0,0.0,...,0.0,0.0,0.2,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
106,0.054545,0.052632,0.116815,0.206553,0.064706,0.421053,0.515464,0.00,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.142857,0.0,0.0,0.0


In [16]:
keep_x_train = x_train
keep_x_test = x_test
keep_y_train = y_train
keep_y_test = y_test

In [17]:
keep_x_train

,Difficulties,retrace_no_corr,gfi,ari,unique_words,text_len,stop_words,about,above,across,...,wrong,yard,yeah,years,yes,yet,you,young,youngster,your
0,0.018182,0.052632,0.160551,0.302301,0.051643,0.421053,0.540146,0.00,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
1,0.020000,0.052632,0.043633,0.109326,0.146199,0.157895,0.520000,0.00,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
2,0.053571,0.052632,0.153351,0.192617,0.074369,0.473684,0.535714,0.25,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.142857,0.0,0.0,0.0
3,0.072727,0.052632,0.059399,0.081080,0.065632,0.421053,0.572727,0.00,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.428571,0.0,0.0,0.0
4,0.050000,1.000000,0.189549,0.286042,0.028382,0.684211,0.540323,0.00,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.428571,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103,0.018519,0.000000,0.138324,0.235141,0.074689,0.368421,0.529412,0.25,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.000000,0.0,0.0,0.0
104,0.018182,0.000000,0.152717,0.263396,0.041793,0.421053,0.564706,0.00,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
105,0.000000,0.052632,0.195747,0.231222,0.076023,0.263158,0.472222,0.25,0.0,0.0,...,0.0,0.0,0.2,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
106,0.054545,0.052632,0.116815,0.206553,0.064706,0.421053,0.515464,0.00,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.142857,0.0,0.0,0.0


In [18]:
keep_x_train = keep_x_train.drop('above', axis=1)
keep_x_test = keep_x_test.drop('above', axis=1)
x_train = x_train.drop('above', axis=1)
x_test = x_test.drop('above', axis=1)

In [19]:
x_test["prediction"] = y_test
x_test
x_train["prediction"] = y_train
x_train

,Difficulties,retrace_no_corr,gfi,ari,unique_words,text_len,stop_words,about,across,action,...,yard,yeah,years,yes,yet,you,young,youngster,your,prediction
0,0.018182,0.052632,0.160551,0.302301,0.051643,0.421053,0.540146,0.00,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,1
1,0.020000,0.052632,0.043633,0.109326,0.146199,0.157895,0.520000,0.00,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,1
2,0.053571,0.052632,0.153351,0.192617,0.074369,0.473684,0.535714,0.25,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.142857,0.0,0.0,0.0,1
3,0.072727,0.052632,0.059399,0.081080,0.065632,0.421053,0.572727,0.00,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.428571,0.0,0.0,0.0,1
4,0.050000,1.000000,0.189549,0.286042,0.028382,0.684211,0.540323,0.00,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.428571,0.0,0.0,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103,0.018519,0.000000,0.138324,0.235141,0.074689,0.368421,0.529412,0.25,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.000000,0.0,0.0,0.0,0
104,0.018182,0.000000,0.152717,0.263396,0.041793,0.421053,0.564706,0.00,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0
105,0.000000,0.052632,0.195747,0.231222,0.076023,0.263158,0.472222,0.25,0.0,0.0,...,0.0,0.2,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0
106,0.054545,0.052632,0.116815,0.206553,0.064706,0.421053,0.515464,0.00,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.142857,0.0,0.0,0.0,0


## Αποθήκευση σε csv αρχεία 

## Saving in csv files 

In [20]:
x_train.to_csv("../../dataset/combined_train.csv", index = False)
x_test.to_csv("../../dataset/combined_test.csv", index = False)